In [1]:
import json
import requests
token = 'secret_ImUJB0y2WWGtECwLayKNfWHG6aSYiRpjl0ekIzvOEzq'
database_id = 'https://www.notion.so/2158a7dfe1cc4c378e1936048e229afa?v=d62129eb66ec4e8984f2021ea204c7fc&pvs=4'

In [10]:
def getMovies():
    url = f'https://api.notion.com/v1/databases/{database_id}/query'
    r = requests.post(url, headers={
    "Authorization": f"Bearer {token}",
    "Notion-Version": "2021–08–16"
    })
    result_dict = r.json()
    movie_list_result = result_dict['results']
    print(movie_list_result)
    movies = []
    for movie in movie_list_result:
        movie_dict = mapNotionResultToMovie(movie)
        movies.append(movie_dict)
    return movies

def mapNotionResultToMovie(result):
    # you can print result here and check the format of the answer.
    movie_id = result['id']
    properties = result['properties']
    genre = properties['Genre']['rich_text'][0]['text']['content']
    name = properties['Name']['title'][0]['text']['content']
    completed = properties['Completed']['checkbox']
    return {
    'genre': genre,
    'name': name,
    'completed': completed,
    'movie_id': movie_id
    }

In [9]:
help(requests.post)

Help on function post in module requests.api:

post(url, data=None, json=None, **kwargs)
    Sends a POST request.
    
    :param url: URL for the new :class:`Request` object.
    :param data: (optional) Dictionary, list of tuples, bytes, or file-like
        object to send in the body of the :class:`Request`.
    :param json: (optional) json data to send in the body of the :class:`Request`.
    :param \*\*kwargs: Optional arguments that ``request`` takes.
    :return: :class:`Response <Response>` object
    :rtype: requests.Response



In [11]:
movies = getMovies()
# json.dumps is used to pretty print a dictionary
print('Movie list:', json.dumps(movies, indent=2))

UnicodeEncodeError: 'latin-1' codec can't encode character '\u2013' in position 4: ordinal not in range(256)